In [ ]:


import numpy as np
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import time

left_img = cv2.imread(Read_image,0)
right_img = cv2.imread(Read_image,0)
left_img = cv2.resize(left_img, None, fx=0.5, fy=0.5)
right_img = cv2.resize(right_img, None, fx=0.5, fy=0.5)
imgHeight, imgWidth = left_img.shape

def NCC(patch1,patch2):
    
    mean_p1 = np.mean(patch1)
    mean_p2 = np.mean(patch2)
    
    std_p1 = np.std(patch1)
    std_p2 = np.std(patch2)
    
    ncc = np.sum((patch1-mean_p1)*(patch2-mean_p2)/(std_p1*std_p2))/(patch1.shape[0]*patch1.shape[1])
    
    return ncc
def BlockMatching(left_img, right_img, blockSize):
    #DispMap will hold the result of the block matching. 
    dispMap = np.zeros((imgHeight, imgWidth))
    #The disparityRange defines how many pixels away from the block's location
    #in the first image to search for a matching block in the other image.
    #We recommend using 30 for the disparityRange.
    disparityRange = 30;
    halfBlockSize = blockSize // 2;
    #For each row 'm' of pixels in the image
    for m in range(imgHeight):
        #Set min/max row bounds for the template and blocks.
        #e.g., for the first row, minr = 1 and maxr = 4
        minr = max(0, m - halfBlockSize)
        maxr = min(imgHeight-1, m + halfBlockSize)
        #For each column 'n' of pixels in the image
        for n in range(imgWidth):
            #Set the min/max column bounds for the template.
            #e.g., for the first column, minc = 1 and maxc = 4
            minc = max(0, n - halfBlockSize)
            maxc = min(imgWidth-1, n + halfBlockSize)
            #Define the search boundaries as offsets from the template location.
            #Limit the search so that we don't go outside of the image. 
            #'mind' is the the maximum number of pixels we can search to the left.
            #'maxd' is the maximum number of pixels we can search to the right.
            #mind = max(-disparityRange, 0 - minc)
            #If we only need to search to the right uncomment the line below and comment out the line above.
            mind = 0; 
            maxd = min(disparityRange, (imgWidth-1) - maxc)
            #Select the block from the right image to use as the template.
            template = right_img[minr:maxr, minc:maxc]
            #Get the number of blocks in this search.
            numBlocks = maxd - mind + 1;
            #Create a vector to hold the block differences.
            blockDiffs = np.zeros((numBlocks, 1))
            
            #Calculate the difference between the template and each of the blocks.
            for i in range(mind, maxd+1):
                #Select the block from the left image at the distance 'i'.
                block = left_img[minr:maxr, (minc + i):(maxc + i)]
                #Compute the index of this block into the 'blockDiffs' vector.
                blockIndex = i - mind
                #Take the sum of absolute differences (SAD) between the template
                #and the block and store the resulting value.
                blockDiffs[blockIndex, 0] = NCC(template, block)

            #Get the index of the closest-matching block.
            bestMatchIndex = blockDiffs.argmax()
            #Convert the index of this block back into an offset.
            #This is the final disparity value produced by basic block matching.
            d = bestMatchIndex + mind
            dispMap[m, n] = d

    return dispMap
  
#Visualize the disparity map
start = time.time()
dispmap = BlockMatching(left_img, right_img, 7)
endt = time.time()
print(endt - start)
plt.figure()
ax = plt.gca()
im = ax.imshow(dispmap, cmap='jet')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)
plt.show()